## Issue #33: Tourigny
Searching the phrase "Tourigny" does not return a result.

The Celery traceback is:

```
07/19/2018 09:59:00 PM [ERROR]: Traceback (most recent call last):
  File "/home/ubuntu/git/GraphSearch/backend/worker.py", line 38, in getGraph
    graphJSON = json.loads(graphSession.get_cy_json(graph_format=kwargs['graph_format'], mode=kwargs['mode']))
  File "/home/ubuntu/git/GraphSearch/backend/lib/GraphSession.py", line 52, in get_cy_json
    return resultGraph.get_graph(graph_format=graph_format)
  File "/home/ubuntu/git/GraphSearch/backend/lib/ResultGraph.py", line 61, in get_graph
    return self.get_cy_json()
  File "/home/ubuntu/git/GraphSearch/backend/lib/ResultGraph.py", line 83, in get_cy_json
    network_stats = self.get_search_stats(n_json)
  File "/home/ubuntu/git/GraphSearch/backend/lib/ResultGraph.py", line 171, in get_search_stats
    top_author_dict = {str(author[0]): author[1] for author in author_counts.most_common(n_most_common)}
  File "/home/ubuntu/git/GraphSearch/backend/lib/ResultGraph.py", line 171, in <dictcomp>
    top_author_dict = {str(author[0]): author[1] for author in author_counts.most_common(n_most_common)}
UnicodeEncodeError: 'ascii' codec can't encode character u'\xe9' in position 5: ordinal not in range(128)
```

Clearly we are dealing with another Unicode encoding issue. 

In [1]:
import sys
sys.path.append('../backend/lib')
sys.path.append('../backend')
from GraphSession import GraphSession
from ResultGraph import ResultGraph
from networkx import generate_gml
import networkx as nx
from networkx.readwrite import json_graph
import json

search_input = "tourigny"#"campbell cancer leukemia" #"nlp genomics"
graphsession = GraphSession(search_input)
citations = graphsession.get_citations_from_fulltext_mongo(search_input, retmax=200)
resultGraph = ResultGraph()
resultGraph.populate_from_cite_dict(citations)
        
resultGraph.extract_by_connectivity(connectivity=1)
resultGraph.extract_by_connectivity(connectivity=0)
        
# Query metadata
metadataList = graphsession.get_metadataList_from_mongo(resultGraph.nodeIds)
resultGraph.add_metadata_to_graph(metadataList)
resultGraph.get_graph(graph_format='cytoscape')

07/20/2018 06:21:01 PM [INFO]: Created new log.
07/20/2018 06:21:01 PM [INFO]: RequestFullTextSearch: Creating request
07/20/2018 06:21:01 PM [INFO]: ConnectEutils: Connecting to eutils API:
 https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&tool=graphsearch&sort=relevance&term=tourigny&retmax=200
07/20/2018 06:21:02 PM [INFO]: MongoSession: Mongo db connection successful
07/20/2018 06:21:02 PM [INFO]: MongoSession: Mongo db connection successful
07/20/2018 06:21:02 PM [INFO]: Max degree: 26


22


'{"graph": {"nodes": [{"data": {"node_col": "#ff7373", "journal": "Indian pacing and electrophysiology journal", "key": "20552060", "authors": "Udink Ten Cate, Wiesner, Trieschmann", "cite_color": "black", "id": "20552060", "group": "Cited", "name": "20552060", "pubDate": "Jun 2010", "title": "Dyssynchronous ventricular activation in asymptomatic wolff-Parkinson-white syndrome: a risk factor for development of dilated cardiomyopathy.", "label": "Indian pacing and electrophysiology journal", "journal_iso": "Indian Pacing Electrophysiol J"}}, {"data": {"node_col": "#ffd4d4", "journal": "Healthcare policy = Politiques de sante", "key": "23115575", "authors": "Breton, L\\u00e9vesque, Pineault", "cite_color": "black", "id": "23115575", "group": "Cited", "name": "23115575", "pubDate": "Nov 2011", "title": "Primary Care Reform: Can Quebec\'s Family Medicine Group Model Benefit from the Experience of Ontario\'s Family Health Teams?", "label": "Healthcare policy = Politiques de sante", "journal

Bug is fixed. Cause was lack of unicode encoding of top_authors dictionary in ResultGraph.py